In [55]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

df = pd.read_csv("Crash_Data.csv")

df = df[[
  'Time',
  'Time of day',
  'Day of week',
  'Month',
  'Year',
  'Dayweek',
  'Crash Type',
  'Road User',
  'Gender',
  'Age',
  'Age Group',
  'State',
  'Speed Limit',
  'Bus Involvement',
  'Articulated Truck Involvement',
  'Christmas Period',
  'Easter Period',
]]

df_with_time = df.copy()
df.drop('Time', axis=1, inplace=True)

df.dropna(axis=0, inplace=True)

df.drop(df[df['Speed Limit'] == 'Unspecified'].index, inplace=True)
df.drop(df[df['Speed Limit'] == '<40'].index, inplace=True)
df.drop(df[df['Speed Limit'] == -9].index, inplace=True)

df['Speed Limit'] = pd.to_numeric(df['Speed Limit'])

df.head(5)

,Time of day,Day of week,Month,Year,Dayweek,Crash Type,Road User,Gender,Age,Age Group,State,Speed Limit,Bus Involvement,Articulated Truck Involvement,Christmas Period,Easter Period
1,Night,Weekend,9,2021,Saturday,Multiple,Pedestrian,Female,28,26_to_39,SA,110,No,No,No,No
5,Night,Weekend,9,2021,Saturday,Multiple,Motorcycle rider,Male,19,17_to_25,Qld,100,No,No,No,No
6,Night,Weekend,9,2021,Saturday,Single,Driver,Female,20,17_to_25,WA,100,No,No,No,No
9,Night,Weekend,9,2021,Saturday,Single,Driver,Male,47,40_to_64,Qld,60,No,No,No,No
10,Night,Weekday,9,2021,Thursday,Single,Driver,Male,24,17_to_25,SA,80,No,No,No,No


In [ ]:
for column in df:
    print(f'{column} -> {set(df[column])}')

--------------------------------------------------------

_**Plotar os dados aqui (3 ou 4 gráficos)**_

--------------------------------------------------------

### Variáveis

*Variáveis categóricas ordinais*
- Age Group
- Month
- Year

*Variáveis categóricas nominais*
- Time of day
- Day of week
- Dayweek
- Crash Type
- Road User
- Gender
- State
- Bus Involvement
- Articulated Truck Involvement
- Christmas Period
- Easter Period

*Variáveis quantitativas (que podem ser usadas em encoding)*
- Time
- Age
- Speed Limit

### Problemas de classificação

**Previsão do Tipo de Acidente** <br />
**Problema:** _Classificar o tipo de acidente como "Single" ou "Multiple" (coluna Crash Type) com base em fatores como Time of day, Day of week, Speed Limit, Road User, etc._

**Previsão de Involvimento de um Ônibus**  <br />
**Problema:** _Classificar se um acidente envolveu um ônibus ou não (coluna Bus Involvement) com base em fatores como Time, Crash Type, Speed Limit, State, etc._

**Previsão do Usuário da Estrada (Pedestre, Motorista, etc.)**  <br />
**Problema:** _Classificar o tipo de usuário da estrada envolvido no acidente (coluna Road User), com base em variáveis como Time, Speed Limit, Crash Type, State, etc._

**Previsão do Estado (Estado da Austrália) onde o Acidente Ocorreu**  <br />
**Problema:** _Classificar em qual estado ocorreu o acidente (coluna State) com base em variáveis como Speed Limit, Crash Type, Road User, Time of day, etc._

### Problemas de regressão

**Previsão da Velocidade Limite (Speed Limit)** <br />
**Problema:** _Prever o valor da velocidade limite na área onde ocorreu o acidente (coluna Speed Limit) com base em variáveis como Time, Crash Type, Day of week, Road User, Age Group, State, etc._

**Previsão da Idade da Pessoa Envolvida** <br />
**Problema:** _Prever a idade (coluna Age) da pessoa envolvida no acidente, com base em variáveis como Time, Crash Type, Road User, State, etc._

**Previsão do Horário do Acidente (Time)** <br />
**Problema:** _Prever o horário exato do acidente (coluna Time) com base em variáveis como Crash Type, Road User, Speed Limit, Dayweek, etc._

**Previsão do Ano do Acidente** <br />
**Problema:** _Prever em que ano ocorreu o acidente (coluna Year) com base em fatores como Crash Type, Age, Road User, Speed Limit, etc._

### Criando o nosso modelo

In [60]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

df_encoded = df.copy() # todos os valores já codificados (categóricos transformados em numéricos através do encoding)

for column in df_encoded:
    if(column not in ['Speed Limit', 'Age', 'Year', 'Month']):
        df_encoded[column] = label_encoder.fit_transform(df_encoded[column])

df_encoded_and_orginal = df.copy() # todos os valores originais em conjunto com os valores codificados

for column in df_encoded_and_orginal:
    if(column not in ['Speed Limit', 'Age', 'Year', 'Month']):
        df_encoded_and_orginal[f'{column} encoded'] = label_encoder.fit_transform(df_encoded_and_orginal[column])

# Organiza a tabela de forma que os valores codificados fiquem ao lado dos valores originais
df_encoded_and_orginal = df_encoded_and_orginal.reindex(columns=sorted(df_encoded_and_orginal.columns))


df_encoded


,Time of day,Day of week,Month,Year,Dayweek,Crash Type,Road User,Gender,Age,Age Group,State,Speed Limit,Bus Involvement,Articulated Truck Involvement,Christmas Period,Easter Period
1,1,1,9,2021,2,0,6,0,28,2,4,110,0,0,0,0
5,1,1,9,2021,2,0,2,1,19,1,3,100,0,0,0,0
6,1,1,9,2021,2,1,0,0,20,1,7,100,0,0,0,0
9,1,1,9,2021,2,1,0,1,47,3,3,60,0,0,0,0
10,1,0,9,2021,4,1,0,1,24,1,4,80,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52838,0,0,1,1989,6,1,4,0,5,0,1,60,1,0,0,0
52839,0,0,1,1989,1,1,4,1,19,1,7,110,0,0,1,0
52840,0,0,1,1989,1,1,4,1,17,1,7,110,0,0,1,0
52841,0,0,1,1989,1,1,4,1,17,1,7,110,0,0,1,0


In [61]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

df_to_predict_crash_type = df_encoded[['Day of week', 'Time of day', 'Road User', 'Crash Type']]

# Separando as features e o target
X = df_to_predict_crash_type.drop('Crash Type', axis=1) 
y = df_to_predict_crash_type['Crash Type']

# Separando os dados entre treino e teste das features e do alvo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Algoritmo de classificação (criação e treinamento)
model = DecisionTreeClassifier()
model.fit(X_train, y_train)

# Utilizando o algoritmo para prever os dados de test
y_pred = model.predict(X_test)

# Métricas de avaliação (accuracy_score e classification_report)
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.6589197611010127
              precision    recall  f1-score   support

           0       0.60      0.71      0.65      6918
           1       0.72      0.62      0.67      8486

    accuracy                           0.66     15404
   macro avg       0.66      0.66      0.66     15404
weighted avg       0.67      0.66      0.66     15404



In [62]:

df_to_predict_crash_type

,Day of week,Time of day,Road User,Crash Type
1,1,1,6,0
5,1,1,2,0
6,1,1,0,1
9,1,1,0,1
10,0,1,0,1
...,...,...,...,...
52838,0,0,4,1
52839,0,0,4,1
52840,0,0,4,1
52841,0,0,4,1
